# yarp planner

This notebook will run through the various initial setup required for running yarp. 

## Clip level 

The following cell will help you in estimating the right clip levels for the various calibrators. Usually the default value should work for the standard flux calibrators like 3C147 and 3C286. This tool will be useful for choosing a right clip level for the phase calibrator using the following steps:

- Choose the number of flagcal loops
- Choose the flux level of the calibrator
- Choose a starting upper limit in units of n-sigma
- Choose the rate at which the cliplevel falls as a function of the "loop-index" using an appropriate beta.
- If you are satisfied with the clip levels feed the above values in the `parameters.yaml` file, except for the flux of the phase calibrator, which is choosen automatically.

In [2]:
# Finds the clip levels for the different flagcal loops

import yaml
import parameters as prms
import numpy as np
import matplotlib.pyplot as plt

def clip_est(fluxscale, uplim, loop, beta):
            return [fluxscale - uplim/loop**beta, fluxscale + uplim/loop**beta]
    
flagcal_loops = 5 # Number of flagcal loops
fluxlevel = 0.84
uplim = 20 #sigma
beta = 0.7 # powerlaw fall wrt loops

for i in range(5):
    print('Clip level',i+1,':',  clip_est(fluxlevel, uplim, i+1, beta))

Clip level 1 : [-19.16, 20.84]
Clip level 2 : [-11.471444133449165, 13.151444133449164]
Clip level 3 : [-8.429261135439395, 10.109261135439395]
Clip level 4 : [-6.738582832551991, 8.41858283255199]
Clip level 5 : [-5.642626386771051, 7.32262638677105]


## Imaging and Self-cal setup

This cell will help you in choosing a source free region in your target field using the FIRST image. This regions helps in estimating rms noise which is used for keeping a track of the improvements in the various selfcal loops. For this: 

- Download the FIRST and/or NVSS images of the target field.
- Create a source-free region file using casaviewer.
- Save the region in the regions folder declared in the `parameters.yaml` file.

In [1]:
from astroquery.skyview import SkyView
from astropy import units as u
from astropy.coordinates import SkyCoord
import subprocess
import yaml

target_ra = 155.51375000 # deg
target_dec = 13.76944444  # deg
FIRST_name = 'AGC203001_FIRST.fits'
NVSS_name = 'AGC203001_NVSS.fits'

with open('parameters.yaml') as file:
    params = yaml.safe_load(file)
outdir = params['general']['outdir']

outdir = '/mnt/work/work/HI_DATA/'
c = SkyCoord(target_ra, target_dec, unit="deg")
paths = SkyView.get_image_list(position=c, survey=['VLA FIRST (1.4 GHz)', 'NVSS'], radius=26*u.arcmin, pixels='1024')


print('Downloading the FIRST image ...', paths[0])
subprocess.run('wget {} -O {}'.format(paths[0], outdir+FIRST_name), shell=True, check=True)

print('Downloading the NVSS image ...', paths[1])
subprocess.run('wget {} -O {}'.format(paths[0], outdir+NVSS_name), shell=True, check=True)

CompletedProcess(args='wget https://skyview.gsfc.nasa.gov/tempspace/fits/skv24643656640318_1.fits -O /mnt/work/work/HI_DATA/AGC203001_NVSS.fits', returncode=0)

## Continuum thermal rms calculator

- Find the total number of visibilities in your data (either the avspc file or full resolution file).
- Give the full bandwidth of your data.
- Use the output thermal rms as an estimate for choosing the various clean thresholds in the imaging-selfcal loops.

Note that this is merely an estimate, true values can be far from this.

In [64]:
# Continuum thermal rms calculator
nvis = (1524844)
bw = ((100*1000)/8192)*10**6 # Hz
Tsys = 45
G = 0.22
interval = 10.7 # secs

rms = (Tsys/(G))*1/(np.sqrt(nvis*bw*interval))*10**3 # 

print('Continuum Thermal rms is', rms, 'mJy')


Continuum Thermal rms is 0.01449373138897218 mJy


## Selfcal planner
- Calculates the SNR per antenna for a given solint.
- Also calculates the minimum solint that can be used for p and aploops.

Reference: https://arxiv.org/abs/1805.05266

In [40]:
# Calculates the SNR per antenna for a given solint.. All the formulas are from the above reference.

#----------------------------------INPUTS----------------------------------------------------

rms = 0.032 # in milli Jy. The final continuum rms of the image. Since in the intermediate selfcal loops 
           # this is usually not achieved, it is better to choose a more conservative value.
    
nants = 29 # total number of working antennas
tonsource = 8.19 # hrs
solint = 30 # mins
peak_flux = 10 # mJy

snr_sc_p = 3 # minimum snr required for p solutions 
snr_sc_ap = 10 # minimum snr required for ap solutions

#-----------------------------------------------------------------------------------------------------------------

rms_ant = rms*np.sqrt(nants - 3) # rms per antenna


print('RMS per antenna:', rms_ant, 'mJy  \n')

rms_sc = rms_ant*np.sqrt((10*3600)/(solint*60))
snr_sc = peak_flux/rms_sc

print('Selfcal RMS for a solint of',solint,'mins:', rms_sc,'mJy \n')
print('Selfcal SNR for a solint of',solint,'mins:', snr_sc,'\n')


# Calculates the minimum solint that can be used for a sensible p and ap solutions.



max_rms_sc_p = peak_flux/snr_sc_p
max_rms_sc_ap = peak_flux/snr_sc_ap


min_solint_p = (rms_ant/max_rms_sc_p)**2*tonsource*60 # in mins
min_solint_ap = (rms_ant/max_rms_sc_ap)**2*tonsource*60 # in mins

print('Minimum solint for phase-only selfcal:', min_solint_p, 'mins \n')
print('Minimum solint for amp-phase selfcal:', min_solint_ap, 'mins \n')

RMS per antenna: 0.1631686244349691 mJy  

Selfcal RMS for a solint of 30 mins: 0.7297122720634482 mJy 

Selfcal SNR for a solint of 30 mins: 13.704031551672333 

Minimum solint for phase-only selfcal: 1.1774730239999998 mins 

Minimum solint for amp-phase selfcal: 13.083033599999997 mins 



## Cube thermal rms calculator

In [48]:
# thermal rms calculator based on number of visibilities

nvis = 1193288 + 274310 #301956 #+113282
ch_width = 100/8192*10**6
c = 299792.458 # km/s
vel_orig = c*(ch_width/(1420.405752*10**6))
vel_cube = 7 #km/s
ratio = vel_cube/vel_orig
#print(ratio)
Tsys = 45
G = 0.22
interval =  10.7 #s

rms = (Tsys/(G))*1/(np.sqrt(nvis*ratio*ch_width*interval))*10**3 # 10.7 is the integration time of each uvdatapoint

print('Per channel thermal rms is', rms, 'mJy')

Per channel thermal rms is 0.28343240948950205 mJy
